## 1. Количество выравниваний (10 баллов).
Вспомим про матрицу про подсчёта динамики с лекции. 
Данная матрица содержит все возможные варианты выравниваний, но некоторые из них эквивалентны, например:

<img src="https://github.com/oxygen311/bioinf-algorithms-2019/blob/master/HW1_Alignments/images/equivalent-alignment-example.png?raw=true" alt="Equivalent Alignment Example" width="400">

Можно заметить, что на данной матрице "жестко" фиксируют выравнивание только диагональные стрелки. 
Горизонтальные же и вертикальные, которые проходят между ними, можно переставлять как угодно.

Без ограничения общности будем считать, что $m < n$. 
Зафиксируем число диагональных стрелок $i: i \leq m$.
При фиксированном $i$ количнство варинтов расположить стрелок равно $\binom n i \binom m i$, для выбора горизонтальных и вертикальныъ координат диагональных стрелок.
$i$ может принимать значения от $0$ до $m$, cледовательно итоговый ответ равен:
$$\sum_{i=0}^m \binom n i \binom m i = \binom {n + m} m$$

## 2. Глобальное выравнивание (Нидлман-Вунш) (14 баллов)
Реализовать алгоритм глобального выравнивания, который на вход получает две последовательности, а выдает их оптимальное выравнивание. 
Фиксированный штраф за несовпадение и гэп и награда 1 за каждое совпадение.

In [1]:
M_INF = -int(1e9)

In [2]:
def recover_solution(p, s1, s2, local=False, local_i_start=None, local_j_start=None):
    if local:
        curr_i, curr_j = local_i_start, local_j_start
        ans_s1, ans_s2 = s1[local_i_start:][::-1] + '|', s2[local_j_start:][::-1] + '|'
    else:
        curr_i, curr_j = len(p) - 1, len(p[0]) - 1
        ans_s1, ans_s2 = "", ""
    
    while not (curr_i <= 0 and curr_j <= 0):
        prev_i, prev_j = p[curr_i][curr_j]
        
        if local and prev_i == -1 and prev_j == -1:
            ans_s1 += '|' + s1[:curr_i][::-1] + ' ' * max(0, curr_j - curr_i)
            ans_s2 += '|' + s2[:curr_j][::-1] + ' ' * max(0, curr_i - curr_j)
        else:
            ans_s1 += s1[curr_i - 1] if curr_i == prev_i + 1 else '-'
            ans_s2 += s2[curr_j - 1] if curr_j == prev_j + 1 else '-'
        
        
        curr_i, curr_j = prev_i, prev_j
    return ans_s1[::-1], ans_s2[::-1]

def argmax_2d_array(bss):
    i_max, j_max, v_max = 0, 0, M_INF
    for i in range(len(bss)):
        for j in range(len(bss[i])):
            if bss[i][j] > v_max:
                v_max = bss[i][j]
                i_max, j_max = i, j
    return i_max, j_max

def alignment(s1, s2, match=1, mismatch=-1, gap=-1, local=False):
    n, m = len(s1), len(s2)
    
    # initialize
    s = [[M_INF  for _ in range(m + 1)] for _ in range(n + 1)]
    p = [[(0, 0) for _ in range(m + 1)] for _ in range(n + 1)]
    s[0][0] = 0
    
   # main algorithm
    for i in range(n + 1):
        for j in range(m + 1):
            if local and s[i][j]:
                s[i][j] = 0
                p[i][j] = (-1, -1)
            # diagonal
            if i > 0 and j > 0 and s[i - 1][j - 1] + (match if s1[i - 1] == s2[j - 1] else mismatch) > s[i][j]:
                s[i][j] = s[i - 1][j - 1] + (match if s1[i - 1] == s2[j - 1] else mismatch)
                p[i][j] = (i - 1, j - 1)
            # horizontal
            if i > 0 and s[i - 1][j] + gap > s[i][j]:
                s[i][j] = s[i - 1][j] + gap
                p[i][j] = (i - 1, j)
            # vertical
            if j > 0 and s[i][j - 1] + gap > s[i][j]:
                s[i][j] = s[i][j - 1] + gap
                p[i][j] = (i, j - 1)
    
    # recover the path
    i_max, j_max = argmax_2d_array(s)
    ans_s1, ans_s2 = recover_solution(p, s1, s2, local=local, local_i_start=i_max, local_j_start=j_max)
    print(ans_s1, ans_s2, sep='\n')

### Тест 1:
Придумайте последовательности одинаковой длины и штрафы так, чтобы в получившемся выравнивании были и несовпадения и гэпы.

In [3]:
alignment('POLYNOMIAL', 'EXPONENTIAL')

--POLYNOMIAL
EXPONEN-TIAL


### Тест 2:
Для последовательностей из теста 1 запустите алгоритм с весами 1 (совпадение), -1 (несовпадение), -0.499 (гэп).

In [4]:
alignment('POLYNOMIAL', 'EXPONENTIAL', match=1, mismatch=-1, gap=-0.499)

--PO--LYN-OMIAL
EXPONE--NT--IAL


## 3. Выравнивание с матрицей весов (6 баллов)
В предыдущей задаче вместо фиксированного штрафа использовать любую матрицу весов. Достаточно (3 + 1) на (3 + 1).

In [5]:
def global_alignment_with_weight_matrix(s1, s2, wss):
    chr_to_ind = {'A': 1, 'B': 2, 'C': 3}
    n, m = len(s1), len(s2)
    
    # initialize
    s = [[M_INF  for _ in range(m + 1)] for _ in range(n + 1)]
    p = [[(0, 0) for _ in range(m + 1)] for _ in range(n + 1)]
    s[0][0] = 0
    
   # main algorithm
    for i in range(n + 1):
        for j in range(m + 1):
            # diagonal
            if i > 0 and j > 0 and s[i - 1][j - 1] + wss[chr_to_ind[s1[i - 1]]][chr_to_ind[s2[j - 1]]] > s[i][j]:
                s[i][j] = s[i - 1][j - 1] + wss[chr_to_ind[s1[i - 1]]][chr_to_ind[s2[j - 1]]]
                p[i][j] = (i - 1, j - 1)
            # horizontal
            if i > 0 and s[i - 1][j] + wss[chr_to_ind[s1[i - 1]]][0] > s[i][j]:
                s[i][j] = s[i - 1][j] + wss[chr_to_ind[s1[i - 1]]][0]
                p[i][j] = (i - 1, j)
            # vertical
            if j > 0 and s[i][j - 1] + wss[0][chr_to_ind[s2[j - 1]]] > s[i][j]:
                s[i][j] = s[i][j - 1] + wss[0][chr_to_ind[s2[j - 1]]]
                p[i][j] = (i, j - 1)
    
    # recover the path
    ans_s1, ans_s2 = recover_solution(p, s1, s2)
    print(ans_s1, ans_s2, sep='\n')

### Тест: 
Придумайте последовательность и матрицу, выровняйте последовательности. Поменяйте в матрице одно число так, чтобы выравнивание поменялось.

In [6]:
wss = [[ 0, -2, -2, -2],
       [-2,  5, -2, -1],
       [-2, -2,  7,  0],
       [-2, -1,  0,  6]]

print('Before change:')
global_alignment_with_weight_matrix('ABC', 'ABB', wss)

wss[2][2] = -10
print()
print('After change:')
global_alignment_with_weight_matrix('ABC', 'ABB', wss)

Before change:
ABC
ABB

After change:
A-BC
AB-B


## 4. Локальное выравнивание (Смит-Ватерман) (6 баллов)
Во 2 задаче найти оптимальное выравнивание всех возможных подслов двух последовательностей, то есть локальное выравнивание. 
В выводе должно быть понятно, где начинается и заканчивается локальное выравнивание.
### Тест:
Придумайте две последовательности, для которых локальное и глобальное выравнивание с одинаковыми штрафами дают разные результаты (на участке локального выравнивания).

In [7]:
print('Global:')
alignment('TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC', 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC')

print()
print('Local:')
alignment('TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC', 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC', 
          local=True)

Global:
---T-CC-CAGT--TATGTCAGGGGACACGAGCATG-CAGAGAC
AATTGCCGCCGTCGT-TTTCA---G-CA-G-TTATGTCAGA-TC

Local:
                  TCC|CAGTTATGTCAG|GGGACACGAGCATGCAGAGAC
AATTGCCGCCGTCGTTTTCAG|CAGTTATGTCAG|ATC


## 5. Афинные гэпы (10 баллов).
Реализуйте алгоритм глобального выравнивания с аффинными гэпами.

In [8]:
def recover_solution_3_leveled(p, s1, s2):
    curr_i, curr_j, curr_l = len(p['m']) - 1, len(p['m'][0]) - 1, 'm'
    ans_s1, ans_s2 = "", ""
    
    while not (curr_i <= 0 and curr_j <= 0):
        prev_i, prev_j, prev_l = p[curr_l][curr_i][curr_j]
        if not (prev_i == curr_i and prev_j == curr_j):
            ans_s1 += s1[curr_i - 1] if curr_i == prev_i + 1 else '-'
            ans_s2 += s2[curr_j - 1] if curr_j == prev_j + 1 else '-'
        
        curr_i, curr_j, curr_l = prev_i, prev_j, prev_l
    return ans_s1[::-1], ans_s2[::-1]

def global_alignment_3_leveled(s1, s2, match=1, mismatch=-1, delta=-1, ro=-1):
    n, m = len(s1), len(s2)
    
    # initialize
    s_d = [[M_INF  for _ in range(m + 1)] for _ in range(n + 1)]
    s_u = [[M_INF  for _ in range(m + 1)] for _ in range(n + 1)]
    s_m = [[M_INF  for _ in range(m + 1)] for _ in range(n + 1)]
    
    p = {'d': [[(0, 0) for _ in range(m + 1)] for _ in range(n + 1)],
         'u': [[(0, 0) for _ in range(m + 1)] for _ in range(n + 1)],
         'm': [[(0, 0) for _ in range(m + 1)] for _ in range(n + 1)]}
    
    s_d[0][0], s_u[0][0] = 0, 0
    
   # main algorithm
    for i in range(n + 1):
        for j in range(m + 1):
            # vertical, one more gap
            if i > 0 and s_u[i - 1][j] + ro > s_u[i][j]:
                s_u[i][j] = s_u[i - 1][j] + ro
                p['u'][i][j] = (i - 1, j, 'u')
            # vertical, start gap
            if i > 0 and s_m[i - 1][j] + delta + ro > s_u[i][j]:
                s_u[i][j] = s_m[i - 1][j] + delta + ro
                p['u'][i][j] = (i - 1, j, 'm')
                
            # horizontal, one more gap
            if j > 0 and s_d[i][j - 1] + ro > s_d[i][j]:
                s_d[i][j] = s_d[i][j - 1] + ro
                p['d'][i][j] = (i, j - 1, 'd')
            # horizontal, start gap
            if j > 0 and s_m[i][j - 1] + delta + ro > s_d[i][j]:
                s_d[i][j] = s_m[i][j - 1] + delta + ro
                p['d'][i][j] = (i, j - 1, 'm')
                
            # diagonal, match or mismatch
            if i > 0 and j > 0 and s_m[i - 1][j - 1] + (match if s1[i - 1] == s2[j - 1] else mismatch) > s_m[i][j]:
                s_m[i][j] = s_m[i - 1][j - 1] + (match if s1[i - 1] == s2[j - 1] else mismatch)
                p['m'][i][j] = (i - 1, j - 1, 'm')
            # end 'u' deletion
            if s_u[i][j] > s_m[i][j]:
                s_m[i][j] = s_u[i][j]
                p['m'][i][j] = (i, j, 'u')
            # end 'u' deletion
            if s_d[i][j] > s_m[i][j]:
                s_m[i][j] = s_d[i][j]
                p['m'][i][j] = (i, j, 'd')
                
    # recover the path
    ans_s1, ans_s2 = recover_solution_3_leveled(p, s1, s2)
    print(ans_s1, ans_s2, sep='\n')

## 5. Афинные гэпы (10 баллов).
Реализуйте алгоритм глобального выравнивания с аффинными гэпами.

Тестовые последовательности: <br>
TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC, <br>
AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC

In [9]:
seq1 = 'TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC'
seq2 = 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC'

## Тест 1. 
Вес совпадения: 1 <br>
Вес несовпадения: -1 <br>
Штраф за открытие гэпа (-p): 0 <br>
Штраф за продолжение (-d): -1

In [10]:
global_alignment_3_leveled(seq1, seq2, match=1, mismatch=-1, delta=0, ro=-1)


--T--CC-CAGT--TATGTCAGGGGACACGAGCATG-CAGAGAC
AATTGCCGCCGTCGT-TTTCAG----CA-G-TTATGTCAGA-TC


## Тест 2. 
Вес совпадения: 1 <br>
Вес несовпадения: -1 <br>
Штраф за открытие гэпа (-p): -100 <br>
Штраф за продолжение (-d): -0.01

In [11]:
global_alignment_3_leveled(seq1, seq2, match=1, mismatch=-1, delta=-100, ro=-0.01)

TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC
AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC


## Тест 3. 
Вес совпадения: 1 <br>
Вес несовпадения: -1 <br>
Штраф за открытие гэпа (-p): +0.5 (награждаем за открытие) <br>
Штраф за продолжение (-d): -0.3

In [12]:
global_alignment_3_leveled(seq1, seq2, match=1, mismatch=-1, delta=0.5, ro=-0.3)

---T----CC--CAGTTATGTCAGGGGACACG--A-GCATGCAGA-GAC
AATTGCCGCCGTC-GTT-T-TCA---G-CA-GTTATG--T-CAGAT--C
